Extracting time-series data from MODIS sensor and products:
 * EVI,
 * LST,
 * NDWI,
 * LAI/FPAR

Annual growing season composites (May-Sept 2000-2021)

maxwell.cook@colorado.edu

In [1]:
# Initialize the GEE API
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=_BZN512fWIiM9y0dKaIWFUwdKcMjElamswtTw8iacnA&tc=NmFdK1UDI1IFsXRvVNf_wxnv6EC_8N77uHWvKEBuhS4&cc=RLjORrahXEA-UzOwQZy2JTHbYK7TN6LXNrYL_1JKVzs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkzzOK-3VvEe_7OvwihshdaUH9_opBuv_-N62H_OR6YqkJQKdqZnB8

Successfully saved authorization token.


In [2]:
# Import the datasets

# Import the MODIS land cover collections.
mod09a1 = ee.ImageCollection("MODIS/061/MOD09A1")
mod11a2 = ee.ImageCollection("MODIS/061/MOD11A2")
mod13a1 = ee.ImageCollection("MODIS/061/MOD13A1")
mod15a2h = ee.ImageCollection("MODIS/061/MOD15A2H")

# Southern Rockies ecoregion (Level III)
srme = ee.FeatureCollection("EPA/Ecoregions/2013/L3")

# Spatial block grid
grid = ee.FeatureCollection("users/maxwell_cook/twensday/Southern_Rockies_Grid250k")
print('Number of features in the asset:', grid.size().getInfo())

print("Success")

Number of features in the asset: 15
Success


In [3]:
# Create a folium map with the spatial grid

import folium

def add_ee_layer(self, ee_object, vis_params, name):
    if isinstance(ee_object, ee.Image):
        map_id_dict = ee.Image(ee_object).getMapId(vis_params)
    elif isinstance(ee_object, ee.FeatureCollection):
        map_id_dict = ee.FeatureCollection(ee_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

fmap = folium.Map(location=[40, -100], zoom_start=4)
fmap.add_ee_layer(grid, {}, 'SRME Block Grid')

# Add a layer control panel to the map
fmap.add_child(folium.LayerControl())

fmap

In [6]:
# Define some of the parameters

# Temporal filters
months = ee.Filter.calendarRange(5, 9, 'month')
years = ee.List.sequence(2000, 2021)
print(years.getInfo())

# Projection information
proj = ee.Projection('EPSG:5070')

print("Success")

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Success


In [7]:
# Create the MODIS stack

# Function for filtering by month and year
def filter_by_month_year(dataset, year, months_filter):
    return dataset.filter(ee.Filter.calendarRange(year, year, 'year')).filter(months_filter)


# Function to compute the median of an image collection and rename bands
def compute_median(collection, bands, prefix):
    return collection.median().rename(list(map(lambda band: prefix + '_md', bands)))


# Function to compute NDWI for an image
def compute_ndwi(image):
    ndwi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename('NDWI')
    return image.addBands(ndwi).select('NDWI')


# Create the time-series stack of median May-Sept
def process_year(y):
    year = ee.Number(y).toInt()
    day = ee.Number(1).toInt()
    month = ee.Number(5).toInt()
    date = ee.Date.fromYMD(year, month, day).millis()

    # Filter the MODIS collections temporally
    mod13a1_filtered = filter_by_month_year(mod13a1, year, months)
    mod11a2_filtered = filter_by_month_year(mod11a2, year, months)
    mod09a1_filtered = filter_by_month_year(mod09a1, year, months)
    mod15a2h_filtered = filter_by_month_year(mod15a2h, year, months)

    # Compute or extract indices EVI, LST, NDWI, LAI/FPAR
    evi = mod13a1_filtered.select('EVI')
    lst = mod11a2_filtered.select('LST_Day_1km')
    ndwi = mod09a1_filtered.map(compute_ndwi)
    lai = mod15a2h_filtered.select('Lai_500m')
    fpar = mod15a2h_filtered.select('Fpar_500m')

    # Compute the growing season median composite
    evi_median = compute_median(evi, ['EVI'], 'EVI')
    lst_median = compute_median(evi, ['LST_Day_1km'], 'LST')
    ndwi_median = compute_median(evi, ['NDWI'], 'NDWI')
    lai_median = compute_median(evi, ['Lai_500m'], 'LAI')
    fpar_median = compute_median(evi, ['Fpar_500m'], 'FPAR')

    # Return the ImageCollection
    return evi_median.addBands([
        lst_median,ndwi_median,lai_median,fpar_median
      ]).set({
        'year': year,
        'system:time_start': date
      }).setDefaultProjection(proj).reproject(proj, None, 500).clip(grid)

# Apply the functions
modis_sr_stack = ee.ImageCollection(years.map(process_year))
print(modis_sr_stack.size().getInfo())

22


In [8]:
# Fetch cell IDs
cell_ids = [ftr['properties']['grid_id'] for ftr in grid.getInfo()['features']]
print(cell_ids)

# Function to export batch
def export_batch(collection, asset_folder, band_name):
    def export_cell(cell_id):
        cell = grid.filter(ee.Filter.eq('grid_id', cell_id)).first()
        cell_id_name = cell.get('grid_id').getInfo()
        # Remove '-' or '+' from cell_id
        cell_id_name = cell_id_name.replace('-', '').replace('+', '')

        for year in years.getInfo():
            year_image = collection.filter(ee.Filter.eq('year', year)).first().clip(cell.geometry())
            export_name = f'Southern_Rockies_MODIS_{band_name}_{year}_{cell_id_name}'

            task = ee.batch.Export.image.toDrive(
                image=year_image,
                description=export_name,
                fileNamePrefix=export_name,
                folder=asset_folder,
                region=cell.geometry(),
                scale=500,
                crs='EPSG:5070',
                maxPixels=1e13
            )
            task.start()

    for cell_id in cell_ids:
      export_cell(cell_id)


# Select the specific bands for each index and call export_batch function
bandNames = ['EVI_md', 'NDWI_md', 'LST_md', 'LAI_md', 'FPAR_md']

modis_export_stack = modis_sr_stack.select(bandNames)

# Select the specific bands for each index and call export_batch function
for band_name in bandNames:
  print(f"Exporting {band_name}")
  export_batch(modis_export_stack.select(band_name), f'twensday/DATA/MODIS/{band_name}/', band_name)

print("Success!")

['-58+20', '-59+19', '-59+20', '-59+21', '-59+22', '-59+23', '-60+19', '-60+20', '-60+21', '-60+22', '-60+23', '-61+20', '-61+21', '-61+22', '-62+21']
Exporting EVI_md
Exporting NDWI_md
Exporting LST_md
Exporting LAI_md
Exporting FPAR_md
